In [3]:
%matplotlib qt

import neuroseries as nts

import numpy as np
import matplotlib.pyplot as plt

from matplotlib import cm

import scipy.stats
import scipy.ndimage
import scipy.signal

import os

import cv2
import datetime

import bk.load 
import bk.plot
import bk.compute

import sklearn.manifold as skm

In [4]:
def pourcent_freezing(interval, freezing_intervals): 
    freezing_intervals_in_interval = freezing_intervals.intersect(interval)
    total_time_freezing = np.sum(freezing_intervals_in_interval.end-freezing_intervals_in_interval.start)
    total_time = interval.end-interval.start
    pourcentage = (total_time_freezing/total_time)*100
    return pourcentage

def all_pourcentages(intervals, freezing_intervals):
    pourcentages = []
    for i in range(len(intervals)):
        interval = intervals.loc[i]
        pourcentages.append(pourcent_freezing(interval, freezing_intervals))
    return pourcentages

def GetStartExp(Exp,Fs = 20000):
    
    if isinstance(Exp[0],(np.bool_,bool)):
        Exp = list(map(int,Exp))
    
    diff_Exp = np.diff(Exp)
    
    t_start = np.where(diff_Exp == 1)[0]    
    return t_start/Fs

In [55]:
import pandas as pd 

def DLC_posRatChelou(filtered = True,start = 0):
    """
    Load position from DLC files (*.h5) and returns it as a nts.TsdFrame
    """
    
    files = os.listdir()
    for f in files:
        if filtered and f.endswith('filtered.h5'): 
            filename = f
            break
        if not filtered and not f.endswith('filtered.h5') and f.endswith('.h5'):
            filename = f
            break
            
    data = pd.read_hdf(filename)
    data = data[data.keys()[0][0]]
    
    TTL = bk.load.digitalin('digitalin.dat')[0,:]
    TTL = TTL[start:]
    tf = bk.compute.TTL_to_times(TTL)
    
    if len(tf)>len(data):
        tf = np.delete(tf,-1)
    
    data.index = tf * 1_000_000
    pos = nts.TsdFrame(data)
    return pos

## Analysis for classical fear conditionning 

In [59]:
context_sess = ['Y:\classical_fear_conditionning\Rat09-20201118\Rat09_201118_180813',
                'Y:\classical_fear_conditionning\Rat10-20201118\Rat10_201118_182234', 
                'Y:\classical_fear_conditionning\Rat11-20201117\Rat11_201117_172705',
                'Y:\classical_fear_conditionning\Rat12-20201117\Rat12_201117_171302', 
                'Y:\classical_fear_conditionning\Rat01-20200630\Rat01_200630_183149',
                'Y:\classical_fear_conditionning\Rat02-20200707\Rat02_200707_172929']
fear_sess = ['Y:\classical_fear_conditionning\Rat09-20201118\Rat09_201118_110003',
             'Y:\classical_fear_conditionning\Rat10-20201118\Rat10_201118_111531',
             'Y:\classical_fear_conditionning\Rat11-20201117\Rat11_201117_112130',
             'Y:\classical_fear_conditionning\Rat12-20201117\Rat12_201117_110146',
            'Y:\classical_fear_conditionning\Rat01-20200630\Rat01_200630_114732',
            'Y:\classical_fear_conditionning\Rat02-20200707\Rat02_200707_114611']

tone_sess = ['Y:\classical_fear_conditionning\Rat09-20201118\Rat09_201118_170529', 
        'Y:\classical_fear_conditionning\Rat10-20201118\Rat10_201118_172028', 
        'Y:\classical_fear_conditionning\Rat11-20201117\Rat11_201117_183012',
        'Y:\classical_fear_conditionning\Rat12-20201117\Rat12_201117_181452', 
        'Y:\classical_fear_conditionning\Rat01-20200630\Rat01_200630_171900', 
        'Y:\classical_fear_conditionning\Rat02-20200707\Rat02_200707_183427' ]


intervals = np.linspace(0,570,20)
starts = intervals[0:-1]
ends = intervals[1:]
intervals = nts.IntervalSet(starts,ends,time_units='s')

digitalin_tone = [3,3,3,3,2,2]
digitalin_shock = [2,2,2,2,1,1]

In [6]:
t_start_exp_tone = [49.9391, 12.74455, 30.36235, 29.62595, 58.95, 17.8116]

ptotal = []
plt.figure()

for fichier,t_s,d_t in zip(tone_sess,t_start_exp_tone,digitalin_tone):
    os.chdir(fichier)
    pos = bk.load.DLC_pos()
    pos.index = pos.index
    tone = bk.compute.tone_intervals(bk.load.digitalin('digitalin.dat')[d_t,:])- t_s * 1_000_000
    speeds = bk.compute.speed(pos,2, columns_to_drop=['e_tail'])
    freeze = bk.compute.freezing_intervals(speeds, 25, mode='multiple_speed', clean=True) - t_s *1_000_000
    pourcents = all_pourcentages(intervals, freeze)
    ptotal.append(pourcents)
#     bk.plot.intervals(freeze,col = 'b')
    plt.plot(np.mean([starts,ends],0), pourcents, '-o',alpha = 0.5)
    
bk.plot.intervals(tone,'orange', alpha=0.5)    
plt.plot(np.mean([starts,ends],0),np.mean(ptotal,0),'*-r')
plt.legend(['Rat09', 'Rat10', 'Rat11', 'Rat12', 'Rat01', 'Rat02','Mean','Tone'])

Z:\Python\Maelle\Amy-Hpc-sleep-dynamics-python\bk\compute.py:161: RuntimeWarning: overflow encountered in square
  v = np.sqrt(x_speed**2 + y_speed**2)


In [60]:
t_start_exp_fear = [37.7209,29.46495,34.7118,25.34385,45.9524,55.00735]

ptotal = []
plt.figure()

for fichier,t_s,d_s in zip(fear_sess,t_start_exp_fear, digitalin_shock):
    os.chdir(fichier)
    pos = bk.load.DLC_pos()
    pos.index = pos.index
    tone = bk.compute.tone_intervals(bk.load.digitalin('digitalin.dat')[d_s+1,:])- t_s * 1_000_000
    shock = bk.compute.tone_intervals(bk.load.digitalin('digitalin.dat')[d_s,:])- t_s * 1_000_000
    speeds = bk.compute.speed(pos,2, columns_to_drop=['e_tail'])
    freeze = bk.compute.freezing_intervals(speeds, 25, mode='multiple_speed', clean=True) - t_s *1_000_000
    pourcents = all_pourcentages(intervals, freeze)
    ptotal.append(pourcents)
#     bk.plot.intervals(freeze,col = 'b')
    plt.plot(np.mean([starts,ends],0), pourcents, '-o',alpha = 0.5)
bk.plot.intervals(shock,'green',alpha = 0.5)    
bk.plot.intervals(tone,'orange', alpha=0.5)    
plt.plot(np.mean([starts,ends],0),np.mean(ptotal,0),'*-r')
plt.legend(['Rat09', 'Rat10', 'Rat11', 'Rat12', 'Rat01', 'Rat02','Mean','Shock','Tone'])

In [62]:
t_start_exp_context = [34,25,36,26,38,58]

ptotal = []
plt.figure()

for fichier,t_s in zip(context_sess,t_start_exp_context):
    os.chdir(fichier)
    pos = bk.load.DLC_pos()
    pos.index = pos.index
    speeds = bk.compute.speed(pos,2, columns_to_drop=['e_tail'])
    freeze = bk.compute.freezing_intervals(speeds, 25, mode='multiple_speed', clean=True) - t_s *1_000_000
    pourcents = all_pourcentages(intervals, freeze)
    ptotal.append(pourcents)
#     bk.plot.intervals(freeze,col = 'b')
    plt.plot(np.mean([starts,ends],0), pourcents, '-o',alpha = 0.5)
plt.plot(np.mean([starts,ends],0),np.mean(ptotal,0),'*-r')
plt.legend(['Rat09', 'Rat10', 'Rat11', 'Rat12', 'Rat01', 'Rat02','Mean'])

In [60]:
# t_start_exp_fear = [37.7209,29.46495,34.7118,25.34385,45.9524,55.00735]

# ptotal = []

# for subsession,t_s,d_s in zip(fear_sess,t_start_exp_fear, digitalin_shock):
#     os.chdir(subsession)
#     pos = bk.load.DLC_pos()
#     tone = bk.compute.tone_intervals(bk.load.digitalin('digitalin.dat')[d_s+1,:])- t_s * 1_000_000 
#     shock = bk.compute.tone_intervals(bk.load.digitalin('digitalin.dat')[d_s,:])- t_s * 1_000_000
#     speeds = bk.compute.speed(pos,2, columns_to_drop=['e_tail'])
#     freeze = bk.compute.freezing_intervals(speeds, 25, mode='multiple_speed', clean=True) - t_s *1_000_000
#     pourcents = all_pourcentages(intervals, freeze)
#     ptotal.append(pourcents)
# sem = scipy.stats.sem(ptotal, axis=0)
# print(sem)

[ 3.18776974  1.48582465  3.23840677  0.64798611 11.55357156 13.12103988
 12.30773501 11.73506083 10.50992606 14.03412223 16.89255854 15.27001727
 16.21988447  7.17805611  7.52125021 17.89187624 18.49912287 14.66015871
 13.17433428]


In [63]:
pos.index

Float64Index([ 11586700.0,  11620025.0,  11653375.0,  11686700.0,  11720025.0,
               11753375.0,  11786700.0,  11820025.0,  11853350.0,  11886700.0,
              ...
              629374050.0, 629407400.0, 629440725.0, 629474050.0, 629507400.0,
              629540725.0, 629574050.0, 629607375.0, 629640725.0, 629674050.0],
             dtype='float64', name='Time (us)', length=18544)

In [15]:
# t_start_exp_fear = [37.7209,29.46495,34.7118,25.34385,45.9524,55.00735]

# ptotal = []
# plt.figure()

# for fichier,t_s,d_s in zip(fear_sess,t_start_exp_fear, digitalin_shock):
#     os.chdir(fichier)
#     pos = bk.load.DLC_pos()
#     pos.index = pos.index
#     tone = bk.compute.tone_intervals(bk.load.digitalin('digitalin.dat')[d_s+1,:])- t_s * 1_000_000
#     shock = bk.compute.tone_intervals(bk.load.digitalin('digitalin.dat')[d_s,:])- t_s * 1_000_000
#     speeds = bk.compute.speed(pos,2, columns_to_drop=['e_tail'])
#     freeze = bk.compute.freezing_intervals(speeds, 25, mode='multiple_speed', clean=True) - t_s *1_000_000
#     pourcents = all_pourcentages(intervals, freeze)
#     ptotal.append(pourcents)
# #     bk.plot.intervals(freeze,col = 'b')
#     plt.plot(np.mean([starts,ends],0), pourcents, '-o',alpha = 0.5)
# sem = scipy.stats.sem(ptotal, axis=0)
# bk.plot.intervals(shock,'green',alpha = 0.5)    
# bk.plot.intervals(tone,'orange', alpha=0.5)    
# plt.errorbar(np.mean([starts,ends],0),np.mean(ptotal,0),sem,'*-r')
# plt.legend(['Rat09', 'Rat10', 'Rat11', 'Rat12', 'Rat01', 'Rat02','Mean','Shock','Tone'])

ValueError: The lengths of the data (19) and the error 3 do not match

In [61]:
for fichier in context_sess:
    os.chdir(fichier)
    pos = bk.load.DLC_pos()
    print(pos.as_units('s').index[-1]-575)
#     f_tone = GetStartExp(bk.load.digitalin('digitalin.dat')[1,:])
#     print(f_tone[f_tone>0])
    
    


34.458774999999946
[  3.0225    3.1475    3.2725  203.08695 353.30415 503.51535]
25.68372499999998
[  3.0225    3.1475    3.2725  203.08695 353.30415 503.51535]
36.234050000000025
[  3.0225    3.1475    3.2725  203.08695 353.30415 503.51535]
26.03037500000005
[  3.0225    3.1475    3.2725  203.08695 353.30415 503.51535]
38.39945
[  3.0225    3.1475    3.2725  203.08695 353.30415 503.51535]
58.11689999999999
[  3.0225    3.1475    3.2725  203.08695 353.30415 503.51535]


## Analysis for PTSD fear conditionning 

In [43]:
neutral_sess = ['Y://fear_conditionning_ptsd\Rat13\Rat13-20201202\Rat13_201202_183404',
               'Y://fear_conditionning_ptsd\Rat14\Rat14-20201203\Rat14_201203_165203',
               'Y://fear_conditionning_ptsd\Rat15\Rat15-20201130\Rat15_201130_164002',
               'Y://fear_conditionning_ptsd\Rat16\Rat16-20201201\Rat16_201201_172742']



context_sess = ['Y://fear_conditionning_ptsd\Rat13\Rat13-20201203\Rat13_201203_154214',
               'Y://fear_conditionning_ptsd\Rat14\Rat14-20201204\Rat14_201204_172709',
               'Y://fear_conditionning_ptsd\Rat15\Rat15-20201201\Rat15_201201_155055',
               'Y://fear_conditionning_ptsd\Rat16\Rat16-20201202\Rat16_201202_171552']

fear_sess = ['Y://fear_conditionning_ptsd\Rat13\Rat13-20201203\Rat13_201203_112541',
            'Y://fear_conditionning_ptsd\Rat14\Rat14-20201204\Rat14_201204_113458',
            'Y://fear_conditionning_ptsd\Rat15\Rat15-20201201\Rat15_201201_115713',
            'Y://fear_conditionning_ptsd\Rat16\Rat16-20201202\Rat16_201202_132308']

tone_sess = ['Y://fear_conditionning_ptsd\Rat13\Rat13-20201203\Rat13_201203_164205',
            'Y://fear_conditionning_ptsd\Rat14\Rat14-20201204\Rat14_201204_161859',
            'Y://fear_conditionning_ptsd\Rat16\Rat16-20201202\Rat16_201202_182329']

latetone_sess = ['Y://fear_conditionning_ptsd\Rat13\Rat13-20201211\Rat13_201211_121503',
                'Y://fear_conditionning_ptsd\Rat14\Rat14-20201211\Rat14_201211_122458',
                'Y://fear_conditionning_ptsd\Rat16\Rat16-20201211\Rat16_201211_134118']

latecontext_sess = ['Y://fear_conditionning_ptsd\Rat13\Rat13-20201211\Rat13_201211_113739',
                   'Y://fear_conditionning_ptsd\Rat14\Rat14-20201211\Rat14_201211_114719',
                   'Y://fear_conditionning_ptsd\Rat15\Rat15-20201211\Rat15_201211_141224',
                   'Y://fear_conditionning_ptsd\Rat16\Rat16-20201211\Rat16_201211_142217']


In [4]:
t_start_exp_neutral = [36.98515,25.79435,36.52745,34.03365]

intervals = np.linspace(0,240,13)
starts = intervals[0:-1]
ends = intervals[1:]
intervals = nts.IntervalSet(starts,ends,time_units='s')

ptotal = []
plt.figure()

for fichier,t_s in zip(neutral_sess,t_start_exp_neutral):
    os.chdir(fichier)
    pos = bk.load.DLC_pos()
    speeds = bk.compute.speed(pos,2, columns_to_drop=['e_tail'])
    freeze = bk.compute.freezing_intervals(speeds, 25, mode='multiple_speed', clean=True) - t_s *1_000_000
    pourcents = all_pourcentages(intervals, freeze)
    ptotal.append(pourcents)
#     bk.plot.intervals(freeze,col = 'b')
    plt.plot(np.mean([starts,ends],0), pourcents, '-o',alpha = 0.5)
plt.plot(np.mean([starts,ends],0),np.mean(ptotal,0),'*-r')
plt.legend(['Rat13', 'Rat14','Rat15','Rat16','Mean'])
plt.title('Pourcent freezing in neutral context before fear conditionning')

Y://fear_conditionning_ptsd\Rat13\Rat13-20201202\Rat13_201202_183404
Y://fear_conditionning_ptsd\Rat14\Rat14-20201203\Rat14_201203_165203
Y://fear_conditionning_ptsd\Rat15\Rat15-20201130\Rat15_201130_164002


ValueError: Length mismatch: Expected axis has 8194 elements, new values have 8195 elements

In [17]:
t_start_exp_fear = [43.6102,61.40995,75.8984,36.58545]

starts = [0,20,40,60,80,100,103,123,138,168,183,213,216]
ends = [20,40,60,80,100,103,123,138,168,183,213,216,240]
intervals = nts.IntervalSet(starts,ends,time_units='s')

ptotal = []
plt.figure()
plt.subplot(3,1,1)

for fichier,t_s in zip(fear_sess,t_start_exp_fear):
    os.chdir(fichier)
    pos = bk.load.DLC_pos()
    tone = bk.compute.tone_intervals(bk.load.digitalin('digitalin.dat')[3,:])- t_s * 1_000_000
    shock = bk.compute.tone_intervals(bk.load.digitalin('digitalin.dat')[2,:])- t_s * 1_000_000
    speeds = bk.compute.speed(pos,2, columns_to_drop=['e_tail'])
    freeze = bk.compute.freezing_intervals(speeds, 30, mode='multiple_speed', clean=True) - t_s *1_000_000
    pourcents = all_pourcentages(intervals, freeze)
    ptotal.append(pourcents)
#     bk.plot.intervals(freeze,col = 'b')
    plt.plot(np.mean([starts,ends],0), pourcents, '-o',alpha = 0.3)
bk.plot.intervals(shock,'green',alpha = 0.5)    
bk.plot.intervals(tone,'orange', alpha=0.5)    
plt.plot(np.mean([starts,ends],0),np.mean(ptotal,0),'*-',color = 'black',linewidth=2)
plt.legend(['Rat13', 'Rat14', 'Rat15', 'Rat16','Mean','Shock','Tone'])
plt.title('Pourcent freezing during fear conditionning')

Text(0.5, 1.0, 'Pourcent freezing during fear conditionning')

In [18]:
t_start_exp_tone = [31.5871,32.76115,5.97]

starts = [0,30,60,75,105,120,150,165,195,210]
ends = [30,60,75,105,120,150,165,195,210,240]
intervals = nts.IntervalSet(starts,ends,time_units='s')

ptotal = []
plt.subplot(3,1,2)

for fichier,t_s in zip(tone_sess,t_start_exp_tone):
    os.chdir(fichier)
    pos = bk.load.DLC_pos()
    tone = bk.compute.tone_intervals(bk.load.digitalin('digitalin.dat')[3,:])- t_s * 1_000_000
    speeds = bk.compute.speed(pos,2, columns_to_drop=['e_tail'])
    freeze = bk.compute.freezing_intervals(speeds, 30, mode='multiple_speed', clean=True) - t_s *1_000_000
    pourcents = all_pourcentages(intervals, freeze)
    ptotal.append(pourcents)
#     bk.plot.intervals(freeze,col = 'b')
    plt.plot(np.mean([starts,ends],0), pourcents, '-o',alpha = 0.3)
    
bk.plot.intervals(tone,'orange', alpha=0.5)    
plt.plot(np.mean([starts,ends],0),np.mean(ptotal,0),'*-',color = 'black',linewidth=2)
plt.legend(['Rat13', 'Rat14', 'Rat16','Mean','Tone'])
plt.title('Pourcent freezing in tone retrieval')

Text(0.5, 1.0, 'Pourcent freezing in tone retrieval')

In [19]:
t_start_exp_context = [23.49815,31.56275,48.84175,24.49015]

intervals = np.linspace(0,240,13)
starts = intervals[0:-1]
ends = intervals[1:]
intervals = nts.IntervalSet(starts,ends,time_units='s')

ptotal = []
plt.subplot(3,1,3)

for fichier,t_s in zip(context_sess,t_start_exp_context):
    os.chdir(fichier)
    speeds = bk.compute.speed(pos,2, columns_to_drop=['e_tail'])
    freeze = bk.compute.freezing_intervals(speeds, 30, mode='multiple_speed', clean=True) - t_s *1_000_000
    pourcents = all_pourcentages(intervals, freeze)
    ptotal.append(pourcents)
#     bk.plot.intervals(freeze,col = 'b')
    plt.plot(np.mean([starts,ends],0), pourcents, '-o',alpha = 0.3)
plt.plot(np.mean([starts,ends],0),np.mean(ptotal,0),'*-',color = 'black',linewidth=2)
plt.legend(['Rat13', 'Rat14', 'Rat15', 'Rat16','Mean'])
plt.title('Pourcent freezing in context retrieval')

Text(0.5, 1.0, 'Pourcent freezing in context retrieval')

In [56]:
t_start_exp_latetone = [54.28285,58.2436,55.1459]

starts = [0,30,60,75,105,120,150,165,195,210]
ends = [30,60,75,105,120,150,165,195,210,240]
intervals = nts.IntervalSet(starts,ends,time_units='s')

ptotal = []
plt.subplot(2,1,1)

for fichier,t_s in zip(latetone_sess,t_start_exp_latetone):
    os.chdir(fichier)
    if fichier == 'Y://fear_conditionning_ptsd\Rat14\Rat14-20201211\Rat14_201211_122458' :
        pos = DLC_posRatChelou(start = 100_000)
    else : pos = bk.load.DLC_pos()
    tone = bk.compute.tone_intervals(bk.load.digitalin('digitalin.dat')[3,:])- t_s * 1_000_000
    speeds = bk.compute.speed(pos,2, columns_to_drop=['e_tail'])
    freeze = bk.compute.freezing_intervals(speeds, 30, mode='multiple_speed', clean=True) - t_s *1_000_000
    pourcents = all_pourcentages(intervals, freeze)
    ptotal.append(pourcents)
#     bk.plot.intervals(freeze,col = 'b')
    plt.plot(np.mean([starts,ends],0), pourcents, '-o',alpha = 0.3)
    
bk.plot.intervals(tone,'orange', alpha=0.5)    
plt.plot(np.mean([starts,ends],0),np.mean(ptotal,0),'*-',color = 'black',linewidth=2)
plt.legend(['Rat13', 'Rat14','Rat16','Mean','Tone'])
plt.title('Pourcent freezing in late tone retrieval')

Text(0.5, 1.0, 'Pourcent freezing in late tone retrieval')

In [57]:
t_start_exp_latecontext = [51.28165,52.87285,62.79435,52.26585]

intervals = np.linspace(0,240,13)
starts = intervals[0:-1]
ends = intervals[1:]
intervals = nts.IntervalSet(starts,ends,time_units='s')

ptotal = []
plt.subplot(2,1,2)

for fichier,t_s in zip(latecontext_sess,t_start_exp_latecontext):
    os.chdir(fichier)
    speeds = bk.compute.speed(pos,2, columns_to_drop=['e_tail'])
    freeze = bk.compute.freezing_intervals(speeds, 30, mode='multiple_speed', clean=True) - t_s *1_000_000
    pourcents = all_pourcentages(intervals, freeze)
    ptotal.append(pourcents)
#     bk.plot.intervals(freeze,col = 'b')
    plt.plot(np.mean([starts,ends],0), pourcents, '-o',alpha = 0.3)
plt.plot(np.mean([starts,ends],0),np.mean(ptotal,0),'*-',color = 'black',linewidth=2)
plt.legend(['Rat13', 'Rat14', 'Rat15','Rat16', 'Mean'])
plt.title('Pourcent freezing in late context retrieval')

Text(0.5, 1.0, 'Pourcent freezing in late context retrieval')

In [46]:
for subsession in latetone_sess:
    os.chdir(subsession)
    f_tone = GetStartExp(bk.load.digitalin('digitalin.dat')[1,:])
    print(f_tone[f_tone>0])

[54.28285]
[58.2436]
[55.1459]


In [28]:
plt.figure()

intervals = np.linspace(0,240,13)
starts = intervals[0:-1]
ends = intervals[1:]
intervals = nts.IntervalSet(starts,ends,time_units='s')

ptotal=[]
t_s = 51.28165
os.chdir(latecontext_sess[0])

pos = bk.load.DLC_pos()
speeds = bk.compute.speed(pos,2, columns_to_drop=['e_tail'])
freeze = bk.compute.freezing_intervals(speeds, 30, mode='multiple_speed', clean=True) - t_s *1_000_000
pourcents = all_pourcentages(intervals, freeze)

plt.subplot(2,1,1)
bk.plot.intervals(freeze,col = 'orange')
pos.index = pos.index -t_s *1_000_000
plt.axhline(25,color = 'red')
plt.plot(np.mean(speeds.as_units('s'),1))
plt.subplot(2,1,2)
plt.plot(np.mean([starts,ends],0), pourcents, '-o') 
plt.ylim([-10,110])

(-10.0, 110.0)